# This Notebook Perform the following steps:

1. Read a Policy PDF document from a UC volume
2. Parsed the PDF documents with PyPDF into text
3. Perform Chunking
4. Create a vector search index from the chunks

In [0]:
%pip install -U --quiet databricks-sdk==0.28.0 databricks-vectorsearch
%pip install --quiet pypdf==4.1.0 tiktoken langchain-text-splitters==0.2.2
%pip install transformers==4.41.1 torch==2.3.0 --quiet
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ./config


# Configuration file

**Prerequisite**: unity catalog needs to be enabled at your workspace

Please change your catalog and schema inside the `config` here to run the notebook on a different catalog

---------------
Current Setup
---------------
Use Catalog: fins_genai
Use Schema: call_center
Use Volumes for policy data: volume_policies
Use Volumes for transcript data: volume_transcripts
Use Vector Search Endpoint name: one-env-shared-endpoint-1



## Helper Functions Shared Across Notebooks

* `get_latest_model_version()`: Return the latest model version
* `index_exists()`: Check whether a vector index already exists
* `endpoint_exists()`: Check whether an endpoint already exists
* `wait_for_vs_endpoint_to_be_ready()`: wait until the vector index endpoint is ready to be queried
* `wait_for_index_to_be_ready()`: wait for vector index to be ready
* `get_endpoint_status()`: collect endpoint status
* `wait_for_model_serving_endpoint_to_be_ready()`: wait for model serving endpoint to be ready

In [0]:
dbutils.widgets.text(name="table_prefix", label="Source Table", defaultValue="policy_docs")

In [0]:
table_prefix = dbutils.widgets.get("table_prefix")
print(f"table_prefix: {table_prefix}")

table_prefix: policy_docs


In [0]:
spark.sql(f"USE CATALOG {catalog};")
spark.sql(f"USE SCHEMA {schema};")

DataFrame[]

# Create a Vector Search Index from PDF Files in a Volume Diretory

In [0]:
tables_config = {
    "raw_files_table_name": f"{table_prefix}_raw_files",
    "parsed_files_table_name": f"{table_prefix}_parsed_files",
    "chunked_files_table_name": f"{table_prefix}_chunked_files"
}

embedding_config = {
    "embedding_endpoint_name": "databricks-gte-large-en",
    "embedding_tokenizer": {
            "tokenizer_model_name": "Alibaba-NLP/gte-large-en-v1.5",
            "tokenizer_source": "hugging_face",
        },
}

chunker_config = {
    "name": "langchain_recursive_char",
    "config": {
        "chunk_size_tokens": 1024,
        "chunk_overlap_tokens": 256
    }
}

## Load Raw PDF from the UC Volume

In [0]:
# Load the raw riles
SOURCE_PATH = f"/Volumes/{catalog}/{schema}/{volume_name_policies}/policy_doc"

raw_files_df = (
    spark.read.format("binaryFile")
    .option("recursiveFileLookup", "true")
    .option("pathGlobFilter", f"*.pdf")
    .load(SOURCE_PATH)
)

# Save to a table
raw_files_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    tables_config["raw_files_table_name"]
)

# reload to get correct lineage in UC
raw_files_df = spark.read.table(tables_config["raw_files_table_name"])

# For debugging, show the list of files, but hide the binary content
display(raw_files_df.drop("content"))

# Check that files were present and loaded
if raw_files_df.count() == 0:
    display(
        f"`{SOURCE_PATH}` does not contain any files.  Open the volume and upload at least file."
    )
    raise Exception(f"`{SOURCE_PATH}` does not contain any files.")

path,modificationTime,length
dbfs:/Volumes/fins_genai/call_center/volume_policies/policy_doc/auto_policy_example.pdf,2025-03-18T05:35:22Z,159906


## Parse PDF with PyPDF

In [0]:
from pypdf import PdfReader
from typing import TypedDict, Dict
import warnings
import io 
from pyspark.sql.functions import udf, col, md5, explode
from pyspark.sql.types import StructType, StringType, StructField, MapType, ArrayType

In [0]:
class ParserReturnValue(TypedDict):
    doc_parsed_contents: Dict[str, str]
    parser_status: str

def parse_bytes_pypdf(
    raw_doc_contents_bytes: bytes,
) -> ParserReturnValue:
    try:
        pdf = io.BytesIO(raw_doc_contents_bytes)
        reader = PdfReader(pdf)

        parsed_content = [page_content.extract_text() for page_content in reader.pages]
        output = {
            "num_pages": str(len(parsed_content)),
            "parsed_content": "\n".join(parsed_content).replace("Allstate", "Autosure"),
        }

        return {
            "doc_parsed_contents": output,
            "parser_status": "SUCCESS",
        }
    except Exception as e:
        warnings.warn(f"Exception {e} has been thrown during parsing")
        return {
            "doc_parsed_contents": {"num_pages": "", "parsed_content": ""},
            "parser_status": f"ERROR: {e}",
        }

# Create UDF
parser_udf = udf(
    parse_bytes_pypdf,
    returnType=StructType(
        [
            StructField(
                "doc_parsed_contents",
                MapType(StringType(), StringType()),
                nullable=True,
            ),
            StructField("parser_status", StringType(), nullable=True),
        ]
    ),
)

In [0]:
# Run the parsing
parsed_files_staging_df = raw_files_df.withColumn("parsing", parser_udf("content")).drop("content")


# Check and warn on any errors
errors_df = parsed_files_staging_df.filter(
    col(f"parsing.parser_status") != "SUCCESS"
)

num_errors = errors_df.count()
if num_errors > 0:
    print(f"{num_errors} documents had parse errors.  Please review.")
    display(errors_df)

# Filter for successfully parsed files
parsed_files_df = parsed_files_staging_df.filter(parsed_files_staging_df.parsing.parser_status == "SUCCESS").withColumn("doc_parsed_contents", col("parsing.doc_parsed_contents")).drop("parsing")

# Write to Delta Table
parsed_files_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tables_config["parsed_files_table_name"])

# reload to get correct lineage in UC
parsed_files_df = spark.table(tables_config["parsed_files_table_name"])

# Display for debugging
print(f"Parsed {parsed_files_df.count()} documents.")

display(parsed_files_df)

Parsed 1 documents.


path modificationTime length doc_parsed_contents dbfs:/Volumes/fins_genai/call_center/volume_policies/policy_doc/auto_policy_example.pdf 2025-03-18T05:35:22Z 159906 Map(parsed_content -> S
A
M
P
L
E
D
O
C
U
M
E
N
TAutosure
Auto Insurance
Policy
Policy:
Issued to:Effective:
AU127-1
S
A
M
P
L
E
D
O
C
U
M
E
N
TAutosure Insurance Company
TABLE OFCONTENTS
General .................................................. 2
When AndWhere ThePolicy Applies ..................... 2
Changes ................................................................... 2
Duty ToReport Autos .............................................. 2
Combining Limits OfTwo OrMore Autos
Prohibited ................................................................. 2
Transfer .................................................................... 2
Cancellation .............................................................. 2
Part 1__Automobile Liability Insurance
Coverages AAandBB..................... 3
Insuring Agreement ................................................. 3
Additional Payments Autosure WillMake .................. 3
Insured Persons ...................................................... 4
Insured Autos .......................................................... 4
Definitions ................................................................ 4
Exclusions__What isnotcovered ......................... 4
Financial Responsibility ........................................... 5
Limits OfLiability ..................................................... 5
IfThere IsOther Insurance ...................................... 5
Assistance AndCooperation ................................... 5
Action Against Autosure ............................................. 6
What ToDoInCase OfAnAuto Accident OrClaim 6
Part 2__Automobile Medical Payments
Coverage CC................................ 6
Insuring Agreement ................................................. 6
Insured Persons ...................................................... 6
Insured Autos .......................................................... 6
Definitions ................................................................ 7
Exclusions__What isnotcovered ......................... 7
Limits OfLiability ..................................................... 7
Unreasonable ofUnnecessary Medical Expenses ..8
IfThere IsOther Insurance ...................................... 8
Assistance AndCooperation ................................... 8
Action Against Autosure ............................................. 8
Proof OfClaim; Medical Reports ............................. 8
Part 3__Automobile Death Indemnity Insurance
Coverage CM................................ 8
Insuring Agreement ................................................. 8
Insured Persons ...................................................... 8
Definitions ................................................................ 9
Exclusions__What isnotcovered ......................... 9Payment OfBenefits; Autopsy ................................. 9
Consent OfBeneficiary ............................................ 9
Proof ofClaim; Medical Reports ............................. 9
Part 4__Automobile Disability Income Protection
Coverage CW ............................... 9
Insuring Agreement ................................................. 9
Insured Persons ...................................................... 9
Definitions ................................................................ 9
Exclusions__What isnotcovered ......................... 9
ToWhom AndWhen Payment IsMade ............... 10
Proof OfClaim; Medical Reports ........................... 10
Part 5__Uninsured Motorists Insurance
Coverage SS.............................. 10
Insuring Agreement ............................................... 10
Insured Persons .................................................... 10
Definitions .............................................................. 11
Exclusions__What isnotcovered .........

## Chunk the parsed text

In [0]:
from functools import partial
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pyspark.sql.functions import when
import tiktoken

In [0]:
class ChunkerReturnValue(TypedDict):
    chunked_text: str
    chunker_status: str


def chunk_parsed_content_langrecchar(
    doc_parsed_contents: str, chunk_size: int, chunk_overlap: int, embedding_config
) -> ChunkerReturnValue:
    try:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=['\nPart '],
            chunk_overlap=0,
            is_separator_regex=True,
            keep_separator=True
        )

        chunks = text_splitter.split_text(doc_parsed_contents)
        return {
            "chunked_text": [doc for doc in chunks],
            "chunker_status": "SUCCESS",
        }
    except Exception as e:
        warnings.warn(f"Exception {e} has been thrown during parsing")
        return {
            "chunked_text": [],
            "chunker_status": f"ERROR: {e}",
        }


chunker_udf = udf(
    partial(
        chunk_parsed_content_langrecchar,
        chunk_size=chunker_config.get('config').get("chunk_size_tokens"),
        chunk_overlap=chunker_config.get('config').get("chunk_overlap_tokens"),
        embedding_config=embedding_config,
    ),
    returnType=StructType(
        [
            StructField("chunked_text", ArrayType(StringType()), nullable=True),
            StructField("chunker_status", StringType(), nullable=True),
        ]
    ),
)

In [0]:
# Run the chunker
chunked_files_df = parsed_files_df.withColumn(
    "chunked",
    chunker_udf("doc_parsed_contents.parsed_content"),
)

# Check and warn on any errors
errors_df = chunked_files_df.filter(chunked_files_df.chunked.chunker_status != "SUCCESS")

num_errors = errors_df.count()
if num_errors > 0:
    print(f"{num_errors} chunks had parse errors.  Please review.")
    display(errors_df)

# Filter for successful chunks
chunked_files_df = chunked_files_df.filter(chunked_files_df.chunked.chunker_status == "SUCCESS").select(
    "path",
    explode("chunked.chunked_text").alias("chunked_text"),
    md5(col("chunked_text")).alias("chunk_id")
)

# add a section meta data column

chunked_files_df = chunked_files_df.withColumn(
    "section",
    when(col("chunked_text").rlike(r"(?i)Part 1"), "Part 1 Automobile Liability Insurance Bodily Injury Coverage Property Damage Coverage")
    .when(col("chunked_text").rlike(r"Part 2"), "Part 2 Automobile Medical Payments Coverage")
    .when(col("chunked_text").rlike(r"Part 3"), "Part 3 Automobile Death Indemnity Insurance Coverage")
    .when(col("chunked_text").rlike(r"Part 4"), "Part 4 Automobile Disability Income Protection Coverage")
    .when(col("chunked_text").rlike(r"Part 5"), "Part 5 Uninsured Motorists Insurance Coverage")
    .when(col("chunked_text").rlike(r"Part 6"), "Part 6 Protection Against Loss To The Auto")
    .otherwise("general")
)

# Write to Delta Table
chunked_files_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    tables_config["chunked_files_table_name"]
)

%md
## Create a Vector Search Index from Chunked Text

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient(disable_notice=True)

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

Endpoint named one-env-shared-endpoint-1 is ready.


In [0]:
# To enable this table as the source of vector search index, we need to enable CDF
spark.sql(f"ALTER TABLE {catalog}.{schema}.{tables_config['chunked_files_table_name']} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

DataFrame[]

In [0]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c

#The table we'd like to index
source_table_fullname = f"{catalog}.{schema}.{tables_config['chunked_files_table_name']}"
# Where we want to store our index
vs_index_fullname = f"{catalog}.{schema}.{tables_config['chunked_files_table_name']}_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  vsc.create_delta_sync_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
    index_name=vs_index_fullname,
    source_table_name=source_table_fullname,
    pipeline_type="TRIGGERED",
    primary_key="chunk_id",
    embedding_source_column='chunked_text', #The column containing our text
    embedding_model_endpoint_name='databricks-bge-large-en' #The embedding endpoint used to create the embeddings
  )
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
else:
  #Trigger a sync to update our vs content with the new data saved in the table
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
  vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).sync()

print(f"index {vs_index_fullname} on table {source_table_fullname} is ready")

Creating index fins_genai.call_center.policy_docs_chunked_files_vs_index on endpoint one-env-shared-endpoint-1...
Waiting for index to be ready, this can take a few min... {'detailed_state': 'PROVISIONING_INDEX', 'message': 'Delta sync Index creation is pending. Check latest status: https://adb-984752964297111.11.azuredatabricks.net/explore/data/fins_genai/call_center/policy_docs_chunked_files_vs_index', 'indexed_row_count': 0, 'ready': False, 'index_url': 'adb-984752964297111.11.azuredatabricks.net/api/2.0/vector-search/indexes/fins_genai.call_center.policy_docs_chunked_files_vs_index'} - pipeline url:adb-984752964297111.11.azuredatabricks.net/api/2.0/vector-search/indexes/fins_genai.call_center.policy_docs_chunked_files_vs_index
index fins_genai.call_center.policy_docs_chunked_files_vs_index on table fins_genai.call_center.policy_docs_chunked_files is ready
